## 流量を管理するモジュール

#### 重力加速度

In [ ]:
def grav():
    return 9.806650

### フーリエの熱伝導  
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

In [6]:
def heatConduction( lam, dtemp, dx2 ):
    return lam * dtemp / dx2

### 気相水分流（水蒸気圧勾配・固体内）  
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

In [7]:
def vapourTransfer_pressure( ldp, dpv, dx2 ):
    return ldp * dpv /dx2

### 液相水分流（水分化学ポテンシャル勾配・固体内）  
定義：$J_l = -\lambda^{'}_{\mu_l}(\nabla・\mu - n_x g)$    
$J_l$：液相水分流量[kg/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [8]:
def liquidTransfer_potential( ldml, dmiu, dx2, nx ):
    return ldml * ( dmiu / dx2 - nx * grav() )